Edición de archivos con Sed
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/bash-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/bash-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 

---

# Introducción

`sed` es un editor de flujos que puede ser usado para extraer, adicionar o reemplazar textos en un archivo. En Ciencia de los datos Resulta particularmente interesante para realizar impresión, sustitución y borrado de textos en archivos planos, que es una tarea común en procesos ETL. Cada comando es representado por una letra y el carácter `/` es usado como un delimitador.


Es posible utilizar el comando 'sed' o 'sed' [Opciones]. Dichas 'opciones' consisten en una secuencia de '-' más un caracter que condiciona funcionalidades predeterminadas. El manual de `sed` puede ser consultado usando `man`.

# Visualización y transformación

Con el siguiente comando, se imprimen los primeros 20 números enteros a un archivo.

In [7]:
seq 20 > out.1

Para imprimir en pantalla todas las líneas que contienen un '1' se usa el comando: 

In [8]:
sed -n '/1/p'  out.1

1
10
11
12
13
14
15
16
17
18
19


La opción `-n` indica que no debe imprimirse en pantalla cada línea leída del archivo `out.1`. La cadena `/1/` indica la expresión regular (en este caso que la línea contenga un `1` y los '/'  son delimitadores). La `p`  al final indica que se imprima la línea. Nóte que esta funcionalidad es similar a la dada por `grep`.

El siguiente comando imprime todas las líneas que contienen un '1' al final:

In [3]:
sed -n '/1$/p'  out.1

1
11


Para imprimir todas las líneas que contienen un '1' al inicio se usaría:

In [3]:
sed -n '/^1/p' out.1

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20


In [14]:
perl -ne 'print if /^[1]/' out.1

1
10
11
12
13
14
15
16
17
18
19


El siguiente comando imprime la tercera línea:

In [5]:
sed -n '3p' out.1

3


El siguiente comando imprime de la linea 3 a la linea 6.

In [6]:
sed -n '3,6 p' out.1 

3
4
5
6


La opción `d` de `sed` indica borrado.

A continuación se requiere imprimir todos los registros excepto los contenidos entre la linea 3 y la linea 6:

In [7]:
sed "3,6 d" out.1 

1
2
7
8
9
10
11
12
13
14
15
16
17
18
19
20


De la línea 15 al final:

In [8]:
sed -n '15,$ p' out.1

15
16
17
18
19
20


La opción `-e` con `sed`  indica expresión (comando) y permite realizar comandos simultaneos; por ejemplo, imprimir todo excepto la posición 1 y 2.

In [9]:
sed -e 1d -e 2d out.1

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


**Ejercicio.--** Para el siguiente archivo, elimine las líneas con datos faltantes, es decir, las identificadas con la cadena `NA`.

In [10]:
cat > out.1 <<EOF
Date, Year, CustomerID, Value
2013-01-12, 2013, 1, 100
2014-05-12, 2014, NA, 100
2013-02-25, 2013, 2, 200
2013-04-04, 2013, 1, 100
2013-06-21, 2013, 2, 200
2014-05-12, 2014, 1, 100
2014-05-12, 2014, NA, 200
2013-02-28, 2013, 1, 100
2013-08-02, 2013, NA, 100
EOF

---

En el archivo generado mediante las siguientes instrucciones:

In [11]:
cat > out.1 <<EOF
FieldA, FieldD, FieldE, FieldG
   2, X, 2X, 2XG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, X, 3X, 3XG
   4, Z, 4Z, 3XG
EOF

se desean cambiar las `X` por `x`.

Para ello, se emplea la opción `s` para indicar que se realizarán sustituciones de texto:

In [12]:
sed 's/X/x/' out.1

FieldA, FieldD, FieldE, FieldG
   2, x, 2X, 2XG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, x, 3X, 3XG
   4, Z, 4Z, 3xG


Note que solo se sustituyeron la primera ocurrencia de cada línea. Si se requieren cambiar todas las ocurrencias en cada línea se usa `g` para indicar sustitución global:

In [13]:
sed 's/X/x/g' out.1

FieldA, FieldD, FieldE, FieldG
   2, x, 2x, 2xG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, x, 3x, 3xG
   4, Z, 4Z, 3xG


La opción `y` es útil para transformar el registro en el orden en el que se indique en la expresión regular; por ejemplo, se requiere cambiar sistemáticamente las X por x y las Y por y.

In [14]:
sed 'y/XY/xy/' out.1 

FieldA, FieldD, FieldE, FieldG
   2, x, 2x, 2xG
   2, y, 2y, 2yG 
   3, y, 3y, 3yG
   3, x, 3x, 3xG
   4, Z, 4Z, 3xG


Se pueden realizar sustituciones de acuerdo con las veces que se encuentre el patrón en la linea; por ejemplo, se requiere sustituir las X por el simbolo # en el archivo pero solo para la segunda columna en donde se encuentren letras X.

In [15]:
sed 's/X/#/2' out.1

FieldA, FieldD, FieldE, FieldG
   2, X, 2#, 2XG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, X, 3#, 3XG
   4, Z, 4Z, 3XG


La opción `2g` indica que se reemplace cuando encuentre el patrón por segunda vez y en adelante.

In [16]:
# Esta opción no funciona en la implementacón de sed en BSD
# sed 's/X/xx/2g' out.1

El simbolo `&` se usa en una expresión regular para indicar la cadena de texto reconocida. Es decir, el símbolo `&` es reemplazado por la cadena de texto reconocida; por ejemplo, se requiere tener las letras X entre llaves para la linea 5.

In [17]:
sed '5 s/X/&}/g' out.1

FieldA, FieldD, FieldE, FieldG
   2, X, 2X, 2XG
   2, Y, 2Y, 2YG 
   3, Y, 3Y, 3YG
   3, X}, 3X}, 3X}G
   4, Z, 4Z, 3XG


---

Sea el siguiente archivo:

In [1]:
cat > out.1 <<EOF
1980-JAN-1+1:0:1.134
1980-JAN-5+1:0:1.12
1982-JAN-13+10:12:42.33
EOF

Se desea formatear la fecha y la hora, es decir, la primera línea:

```
1980-JAN-1+1:0:1.134
```

debe cambiarse por:

```
1980-JAN-01 01:00:01
```


El primer paso consiste en agregar el cero a los números de día con un solo dígito.

In [5]:
sed 's/-\([0-9]\)+/-0\1+/' out.1 > out.2
cat out.2

1980-JAN-01+1:0:1.134
1980-JAN-05+1:0:1.12
1982-JAN-13+10:12:42.33


La explicación del comando anterior es la siguiente. El patrón de entrada está conformado por los siguientes elementos:
* El caracter '-'.
* Un dígito entre 0 y 9 (patrón `[0-9]`). Las secuencias `\(` y `\)` especifican que el dígito reconocido debe recordarse. Pueden existir varias cadenas a recordar; la primera cadena es `\1`, la segunda cadena es `\2` y así sucesivamente.
* El caracter `+`.

El patrón de salida indica que:
* Se imprime el caracter `-`.
* Luego el caracter `0`.
* A continuación el dígito reconocido `\1`.


Se reemplaza el `+` por un espacio en blanco.

In [20]:
sed 's/+/ /' out.2 > out.3
cat out.3

1980-JAN-01 1:0:1.134
1980-JAN-05 1:0:1.12
1982-JAN-13 10:12:42.33


Se agrega el `0` a las horas. La expresión regular indica que es un espacio en blanco seguido de un dígito, seguido de `:`.

In [21]:
sed 's/ \([0-9]\):/ 0\1:/' out.3 > out.4
cat out.4

1980-JAN-01 01:0:1.134
1980-JAN-05 01:0:1.12
1982-JAN-13 10:12:42.33


Se agrega el `0` a los minutos.

In [22]:
sed 's/:\([0-9]\):/:0\1:/' out.4 > out.5
cat out.5

1980-JAN-01 01:00:1.134
1980-JAN-05 01:00:1.12
1982-JAN-13 10:12:42.33


Se agrega el `0` a los segundos.

In [23]:
sed 's/:\([0-9]\)\./:0\1./' out.5 > out.6
cat out.6

1980-JAN-01 01:00:01.134
1980-JAN-05 01:00:01.12
1982-JAN-13 10:12:42.33


Se elimina la parte decimal de los segundos

In [24]:
sed 's/\.[0-9][0-9]*//' out.6 > out.7
cat out.7

1980-JAN-01 01:00:01
1980-JAN-05 01:00:01
1982-JAN-13 10:12:42


La notación `\.[0-9][0-9]*` indica que el patrón es un punto (`\.`) seguido de un dígito (`[0-9]`), seguido de cero, uno o más dígitos (`[0-9]*`).

---

Sea el siguiente archivo:

In [25]:
cat > out.1 <<EOF
Angelica Montes 2541122 SI
Diana Castro 4223647 SI
Gerardo Lopez 5447996 NO

EOF


* La función `sed` junto con la opción `i`, permiten agregar lineas **antes** de un texto que es reconocido mediante expresión regular. De la misma forma, la opción `a` permite agregar lineas **después** del texto reconocido.


* La función `sed` junto con la opción `c`, permiten cambiar la información de un registro que cumpla un patrón específico. 

Se requiere incluir antes del registro `Diana Castro`, la siguiente información:

`Camilo Fernandez 6114432 SI`

In [26]:
#Esta opción no funciona en sed de BSD
sed '/Diana/ i\ Camilo Fernandez 6114432 SI/' out.1

Angelica Montes 2541122 SI
 Camilo Fernandez 6114432 SI/
Diana Castro 4223647 SI
Gerardo Lopez 5447996 NO



Se requiere cambiar el registro `Diana Castro` por la siguiente información:

`Camilo Fernandez 6114432 SI`

In [27]:
#Esta opción no funciona en sed de BSD
sed '/Diana/ c Camilo Fernandez 6114432 SI' out.1

Angelica Montes 2541122 SI
Camilo Fernandez 6114432 SI
Gerardo Lopez 5447996 NO



---

Sea el siguiente archivo:

In [28]:
cat > out.1 <<EOF
Date, Year, CustomerID, Value
2013-01-12, 2013, 1, 100
2014-05-12, 2014, 1, 100
2013-02-25, 2013, 2, 200
2013-04-04, 2013, 1, 100
2013-06-21, 2013, 2, 200
2014-05-12, 2014, 12, 100
2014-05-12, 2014, 2, 200
2013-02-28, 2013, 11, 100
2013-08-02, 2013, 1, 100
EOF

Se desea agregar un nuevo campo llamado `Year-CoustomerID` que contiene una clave compuesta conformada por la concatenación de estos dos campos; por ejemplo, el valor para el primer registro sería `2013-1`. El siguiente comando hace el cambio del reglón dos en adelante:

In [29]:
sed 's/ \([0-9][0-9][0-9][0-9]\), \([0-9]*\)/ \1, \2, \1-\2/' out.1 > out.2
cat out.2

Date, Year, CustomerID, Value
2013-01-12, 2013, 1, 2013-1, 100
2014-05-12, 2014, 1, 2014-1, 100
2013-02-25, 2013, 2, 2013-2, 200
2013-04-04, 2013, 1, 2013-1, 100
2013-06-21, 2013, 2, 2013-2, 200
2014-05-12, 2014, 12, 2014-12, 100
2014-05-12, 2014, 2, 2014-2, 200
2013-02-28, 2013, 11, 2013-11, 100
2013-08-02, 2013, 1, 2013-1, 100


Para realizar el cambio en la primera línea (el encabezado) se usaría el siguiente comando:

In [30]:
sed 's/\([a-zA-Z]*\), \([a-zA-Z]*\), \([a-zA-Z]*\), \([a-zA-Z]*\)/\1, \2, \3, \2-\3, \4/' out.2

Date, Year, CustomerID, Year-CustomerID, Value
2013-01-12, 2013, 1, 2013-1, 100
2014-05-12, 2014, 1, 2014-1, 100
2013-02-25, 2013, 2, 2013-2, 200
2013-04-04, 2013, 1, 2013-1, 100
2013-06-21, 2013, 2, 2013-2, 200
2014-05-12, 2014, 12, 2014-12, 100
2014-05-12, 2014, 2, 2014-2, 200
2013-02-28, 2013, 11, 2013-11, 100
2013-08-02, 2013, 1, 2013-1, 100


---

**Ejercicio.--** Convierta el formato de las fechas de `D/M/Y` a `YYYY-MM-DD` en los archivos `order*`.

**Ejercicio.--** Agregue dos nuevos campos al archivo `order`; el primer campo corresponde al mes y el segundo al año. 

**Ejercicio.--** Elimine del archivo `person` todos los registros de las personas  que hayan nacido después del año `1980`.

---

### Borrado de los archivos temporales creados

In [31]:
rm out.*

---

Sed
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/bash-for-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/bash-for-analytics/tree/master/) para explorar el repositorio usando `nbviewer`. 